# Example Data Science Workflow

This workbook will go through a simple analysis for NOAA weather data. This will include:

* importing data
* checking the validatity and integretity of the import
* exploring the data
* visualizing the data
* explore a specific research question
* plot some geographic data

Print out the cheatsheet to help you- https://www.rstudio.com/wp-content/uploads/2015/02/data-wrangling-cheatsheet.pdf 

```
# NOTE: THERE ARE INTENTIONAL OMMISSIONS IN THE CODE BELOW. YOU WILL HAVE TO FIX THE CODE TO MAKE IT WORK.
```

## Read in the data:

This data was retreived from Google's BigQuery and exported to a csv.

In [ ]:
library(readr) # This library lets us read CSVs simply
library(tidyverse) # This library loads a variety of tools we need

# This is the query to pull back the data from Google's BigQuery (note it is using standard SQL dialect)
#
# SELECT b.name, b.country, b.state, b.call, b.lat, b.lon, b.elev, b.begin, b.end, a.wban, 
#        a.stn, a.year, a.mo, a.da, a.temp, a.count_temp, a.dewp, a.count_dewp, a.stp, a.count_stp, 
#        a.visib, a.count_visib, a.wdsp, a.count_wdsp, a.gust, a.max, a.flag_max, a.min, a.flag_min, 
#        a.prcp, a.flag_prcp, a.sndp, a.fog, a.rain_drizzle, a.snow_ice_pellets, a.hail, a.thunder, 
#        a.tornado_funnel_cloud
# FROM `bigquery-public-data.noaa_gsod.gsod*` a
# JOIN `bigquery-public-data.noaa_gsod.stations` b ON a.stn=b.usaf AND a.wban=b.wban
# WHERE state LIKE "AZ"

azgsod <- read_csv("./data/azgsod.zip") 

## Check the integrity of the data

Just because your code reads data successfully, does not mean it read it correctly. Checking the import to catch any issues will save you time later.

In [ ]:
azgsod # look at the dataframe

In [ ]:
?lapply # let's use the lapply function to apply a function that works on single columns to the dataframe.

In [ ]:
lapply(azgsod, typeof) # look at the data types

In [ ]:
lapply(azgsod, range) # does the data seem correct looking at the min and max values?

In [ ]:
# CLEANING THE DATA
# remove old school convention of setting missing data to nines
# azgsod$dewp[azgsod$dewp == 9999.9] <- NA # boolean mask

azgsod <- read_csv("./data/azgsod.zip", na = c('9999.9','999.9'))

## Explore the data

In [ ]:
# let's count the number of observations by station

azgsod %>%
  count(name) 

In [ ]:
# Now let's arrange the output by the count in descending order (using the "n" column) in the pipeline above. 
# Can you figure out how to do this with the code below using the desc function added to the pipeline above?

arrange(desc())

In [ ]:
# look at observations by year

azgsod %>%
  count(year) %>%
  print(n=100) 


## Research question

Can we see the effects of global warming using this dataset?

In [ ]:
# Let's start to visualize the data by looking at temp by year. This will take some time given the dataset is large.
azgsod %>%
  ggplot(aes(x=year, y=temp)) + 
  geom_point()

In [ ]:
# A better, faster way to plot large datasets is to use geom_boxplot().

azgsod %>%
  ggplot(aes(x=year, y=temp)) +
  geom_boxplot()

In [ ]:
# Note that the boxplot didn't plot each year which is what I originally intended. 
# To fix this, use as.factor() to change the year variable to a "factor" datatype above.
# Let's add a single column(variable) for every year, month, and day.

azgsod %>%
mutate(yrmoda = ISOdatetime(.$year, .$mo, .$da, 0, 0, 0)) %>%
  ggplot(aes(yrmoda, temp)) + 
  geom_line()

In [ ]:
# Let's focus the time period to look at the pattern
# Instead of looking at all the datapoints, let's group by year.

azgsod %>% 
  group_by(year) %>% 
  summarise(mean_temp =mean(temp)) %>% 
  ggplot(aes(x=year, y=mean_temp)) + 
  geom_point()

In [ ]:
# Let's look at the number of data points by station

azgsod %>%
  group_by(name) %>%
  summarise(count_temp = n()) %>%
  ggplot(aes(count_temp)) + 
  geom_histogram()

In [ ]:
# Let's look at the top stations

azgsod %>%
  group_by(name) %>%
  summarise(count_temp = n()) %>%
  arrange(desc(count_temp))

In [ ]:
# Now, let's focus the analysis and look at just a single station 'DAVIS-MONTHAN AFB AIRPORT'

# Add the filter parameter below for our target station

azgsod %>% 
  filter(name == 'DAVIS-MONTHAN AFB AIRPORT') %>% # this line is broken
  count(year) %>%
  print(n=100)

What does this output tell us about the data? Is the time series complete?

In [ ]:
# Let's add our datetime var and assign our focused data to its own dataframe
# read the code below to describe each action R will take 

azgsod %>% 
  filter(name == ?) %>%
  mutate(yrmoda = ISOdatetime(.$year, .$mo, .$da, 0, 0, 0)) -> davis

In [ ]:
# Now, let's plot it

davis %>%
  select(?, temp) %>%
  ggplot(aes(yrmoda, temp)) + 
  geom_point()

How can we change the plot above to see what the pattern looks like in a given year.

In [ ]:
# Let's look at mean monthly temp data

davis %>%
  group_by(year, mo) %>%
  summarise(mean= ?) -> davis_monthly

In [ ]:
# Let's plot some max temps

davis %>%
  group_by(year) %>%
  summarise(mean_max_temp = ?) %>%
  ggplot(aes(year, mean_max_temp)) + 
  geom_point() 

Add a trendline to the plot above using ```stat_smooth()```.

In [ ]:
# Now, let's just look at some specific months like July

davis %>%
  ? %>%
  group_by(year,mo) %>%
  summarise(mean_max_temp = max(temp)) %>%
  ggplot(aes(year, mean_max_temp)) + 
  geom_point() + 
  stat_smooth()

## Let's create a simple linear model

In [ ]:
library(modelr) # this is a simple modeling library

davis_mod <- lm(mean ~ as.factor(mo), data= davis_monthly)

summary(davis_mod)

In [ ]:
# plot model predictions

davis_monthly %>% 
  modelr::add_predictions(davis_mod) %>%
  ggplot(aes(year + mo/12, pred)) + 
  geom_line()

In [ ]:
# plot residuals

davis_monthly %>% 
  modelr::add_residuals(davis_mod) %>%
  ggplot(aes(year + mo/12, resid)) + 
  geom_point()

In [ ]:
# Now let's use an additive seasonal decomposition to look at the trend minus the seasonality

ts_davis_mo = ts(davis_monthly$mean, frequency = 12)
decompose_davis_mo = decompose(ts_davis_mo, "additive")



plot(as.ts(decompose_davis_mo$trend))
plot(as.ts(decompose_davis_mo$seasonal))
plot(as.ts(decompose_davis_mo$random))
plot(decompose_davis_mo)

In [ ]:
# What is the ts function?
?ts 

In [ ]:
?decompose # what time period is the code above using to remove seasonality?

## As a final example, let's look at how to bring maps into our plots

In [ ]:
library(maps) # the maps library to plot geo data

# create a base plot to draw on
p <- ggplot() +
  coord_fixed() +
  xlab("") +
  ylab("")

# get the state data to draw state map
us_map <- map_data("state")
az_map <- subset(us_map, us_map$region=="arizona")

# create the base az map 
base_az_map <- p + geom_polygon(data=az_map, aes(x=long, y=lat, group=group), 
                                     colour="light blue", fill="light blue")

# filter for just the unique stations lat and longs
azgsod %>%
  select(name, lat, lon) %>%
  unique() -> az_station_locs

# add the station locations to the base plot and base Arizona map
base_az_map +
  geom_point(data=az_station_locs, aes(x=lon, y=lat), color="dark blue", size=3, alpha=.2)

# What does this plot show you?
# How might you want to add to it to better understand the data?

